In [1]:
# Restart kernel first. Then run:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # reduce fragmentation (set before torch import)

import gc, torch
gc.collect()
torch.cuda.empty_cache()


In [2]:
!nvidia-smi

Sat Oct 11 01:24:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 19.0 MB/s eta 0:00:00


In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 32.0 MB/s eta 0:00:00


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import evaluate
import matplotlib.pyplot as plt

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
device="cuda" if torch.cuda.is_available() else "cpu"
model_cpkt="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model_cpkt)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model_cpkt).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
model_pegasus.gradient_checkpointing_enable()


In [9]:
!wget https://github.com/AtharvTungatkar/Text-Summarizer/blob/main/Data/summarizer-data.zip
!wget https://raw.githubusercontent.com/AtharvTungatkar/Text-Summarizer/main/Data/summarizer-data.zip -O summarizer-data.zip
!unzip summarizer-data.zip -d summarizer-data

--2025-10-11 01:28:06--  https://github.com/AtharvTungatkar/Text-Summarizer/blob/main/Data/summarizer-data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip     [   <=>              ] 180.67K   297KB/s    in 0.6s    

2025-10-11 01:28:07 (297 KB/s) - ‘summarizer-data.zip’ saved [185007]

--2025-10-11 01:28:08--  https://raw.githubusercontent.com/AtharvTungatkar/Text-Summarizer/main/Data/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===========

In [10]:
dataset_samsum=load_from_disk('summarizer-data/samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [11]:
def convert_examples_to_features(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)

  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
  }

In [12]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [14]:
# Shuffle and select only 50% of the dataset
from datasets import DatasetDict

# Halve each split
dataset_samsum_pt_reduced = DatasetDict({
    "train": dataset_samsum_pt["train"].shuffle(seed=42).select(range(len(dataset_samsum_pt["train"])//2)),
    "validation": dataset_samsum_pt["validation"].shuffle(seed=42).select(range(len(dataset_samsum_pt["validation"])//2))
})

# Optional: test split if you have one
# "test": dataset_samsum_pt["test"].shuffle(seed=42).select(range(len(dataset_samsum_pt["test"])//2))


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [16]:
# Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)


In [17]:
from transformers import TrainingArguments, Trainer

trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=2,warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    fp16=True,  # mixed precision
    gradient_accumulation_steps=4,
    logging_steps=200,
    save_steps=500,
    save_total_limit=1,
    report_to=[]
)

In [18]:
trainer=Trainer(model=model_pegasus,args=trainer_args,processing_class=tokenizer,data_collator=seq2seq_data_collator,
                train_dataset=dataset_samsum_pt_reduced['train'],
                eval_dataset=dataset_samsum_pt_reduced['validation'])

In [19]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
200,2.547600
400,1.875700
600,1.769900
800,1.709100
1000,1.681100
1200,1.604300
1400,1.603600
1600,1.592700
1800,1.583500
2000,1.448500


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3684, training_loss=1.5946078781972879, metrics={'train_runtime': 6118.5522, 'train_samples_per_second': 2.408, 'train_steps_per_second': 0.602, 'total_flos': 5532596343619584.0, 'train_loss': 1.5946078781972879, 'epoch': 2.0})

In [20]:
## Save Model and tokenizer
model_pegasus.save_pretrained("pegasus-samsum-model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [21]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,device=device,column_text='article',column_summary='highlights'):
  article_batches=list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches=list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
    inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")

    summaries=model.generate(input_ids=inputs['input_ids'].to(device),
                             attention_mask=inputs['attention_mask'].to(device),
                             length_penalty=0.8,num_beams=8,max_length=128)

    decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True) for s in summaries]
    metric.add_batch(predictions=decoded_summaries,references=target_batch)
  score=metric.compute()
  return score


In [22]:
rouge_names=['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric=evaluate.load('rouge')

In [28]:
score=calculate_metric_on_test_ds(
    dataset_samsum_pt['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2,column_text='dialogue',     # override default 'article'
    column_summary='summary'
)
rouge_dict=dict((rn,score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict,index=[f'pegasus'])

100%|██████████| 5/5 [00:36<00:00,  7.33s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.121029,0.008929,0.102336,0.102575


In [30]:
# Prediction

gen_kwargs={'length_penalty':0.8,'num_beams':8,'max_length':128}
# max length to change output size
sample_text=dataset_samsum['test'][0]['dialogue']
reference=dataset_samsum['test'][0]['summary']

In [31]:
sample_text

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [32]:
reference

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [34]:
tokenizer=AutoTokenizer.from_pretrained('/content/tokenizer')
pipe=pipeline('summarization',model='pegasus-samsum-model',tokenizer=tokenizer)

Device set to use cuda:0


In [35]:
print(pipe(sample_text,**gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Amanda can't find Betty's number. Larry called her last time they were at the park together. She will text him if she wants to talk to him.


In [ ]:
# You can see that the original summary and model summary is same. Hence model is working well.